Retriever And Chain With Langchain

In [3]:
#PDF reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("Soham_IIITN(3).pdf")
docs=loader.load()
docs

[Document(page_content='Soham Deshpande\n+91-9307005165 |sohamdeshpande242@gmail.com |LinkedIn |GitHub\nEducation\nIndian Institute Of Information Technology, Nagpur Maharashtra, India\nElectronics and Communication Engineering (CGPA : 8.34) Dec 2021 – May 2025\nAchievements\nCompetitive Programming\n•Rated 1694 onLeetCode (Top 16.66%) , and 3 Star onCodeChef by achieving max rating of 1694.\n•I achieved an impressive Global Rank of 17 in CodeChef Starters 108, surpassing more than 21k participants.\n•Solved over 750+ DSA/CP Problems , showcasing proficiency in problem-resolution and algorithm design.\nHackathons and Open Source Github\n•Top position in Version-Beta, MANIT Bhopal Hackathon, creating the most practical regional solution.\n•Regional winners of Smart India Hackathon (SIH) , demonstrating strong problem-solving and technical skills.\n•GSSOC Open Source Contributor : Added BVPS and GST calculation functions to a fintech API.\nProjects\nWhatsApp-Chat-Analyzer (Link )|Python,

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(page_content='Soham Deshpande\n+91-9307005165 |sohamdeshpande242@gmail.com |LinkedIn |GitHub\nEducation', metadata={'source': 'Soham_IIITN(3).pdf', 'page': 0}),
 Document(page_content='Education\nIndian Institute Of Information Technology, Nagpur Maharashtra, India', metadata={'source': 'Soham_IIITN(3).pdf', 'page': 0}),
 Document(page_content='Electronics and Communication Engineering (CGPA : 8.34) Dec 2021 – May 2025\nAchievements', metadata={'source': 'Soham_IIITN(3).pdf', 'page': 0}),
 Document(page_content='Achievements\nCompetitive Programming', metadata={'source': 'Soham_IIITN(3).pdf', 'page': 0}),
 Document(page_content='•Rated 1694 onLeetCode (Top 16.66%) , and 3 Star onCodeChef by achieving max rating of 1694.', metadata={'source': 'Soham_IIITN(3).pdf', 'page': 0})]

In [5]:
documents=text_splitter.split_documents(docs)
documents


[Document(page_content='Soham Deshpande\n+91-9307005165 |sohamdeshpande242@gmail.com |LinkedIn |GitHub\nEducation', metadata={'source': 'Soham_IIITN(3).pdf', 'page': 0}),
 Document(page_content='Education\nIndian Institute Of Information Technology, Nagpur Maharashtra, India', metadata={'source': 'Soham_IIITN(3).pdf', 'page': 0}),
 Document(page_content='Electronics and Communication Engineering (CGPA : 8.34) Dec 2021 – May 2025\nAchievements', metadata={'source': 'Soham_IIITN(3).pdf', 'page': 0}),
 Document(page_content='Achievements\nCompetitive Programming', metadata={'source': 'Soham_IIITN(3).pdf', 'page': 0}),
 Document(page_content='•Rated 1694 onLeetCode (Top 16.66%) , and 3 Star onCodeChef by achieving max rating of 1694.', metadata={'source': 'Soham_IIITN(3).pdf', 'page': 0}),
 Document(page_content='•I achieved an impressive Global Rank of 17 in CodeChef Starters 108, surpassing more than 21k', metadata={'source': 'Soham_IIITN(3).pdf', 'page': 0}),
 Document(page_content='mor

In [6]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents[:15],OllamaEmbeddings())

In [7]:
db

In [8]:

query="What are the Achievements"
result=db.similarity_search(query)
result[0].page_content

'Electronics and Communication Engineering (CGPA : 8.34) Dec 2021 – May 2025\nAchievements'

In [9]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [10]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [11]:
#chain 
#Create Stuff Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)

In [12]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022A506D53D0>)

Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 

Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response

In [14]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [18]:
response=retrieval_chain.invoke({"input":"What are the all Achievements"})

In [19]:
response['answer']

"Based on the provided context, the user has achieved the following achievements:\n\n1. Rated 1694 on LeetCode (Top 16.66%)\n2. Max rating of 1694 on CodeChef\n\nTherefore, the user's all achievements are:\n\n1. Rated 1694 on LeetCode\n2. Max rating of 1694 on CodeChef"